In [164]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np

from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import accuracy_score, f1_score, make_scorer, recall_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_curve, recall_score, accuracy_score, precision_score, plot_confusion_matrix, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


In [165]:
#Importar datasets
df = pd.read_csv('C:/Users/jordi/Documents/GitHub/SupplyChainResilience-Datathon-2022-UPC/JOINED_DATA/df_joined.csv')
test_kaggle = pd.read_csv('C:/Users/jordi/Documents/GitHub/SupplyChainResilience-Datathon-2022-UPC/JOINED_DATA/PreTest.csv')

In [166]:
# Clean test dataset

test_kaggle = test_kaggle.drop(['product_id','order_id'],axis=1)
test_kaggle = test_kaggle.dropna()

test_kaggle = pd.get_dummies(test_kaggle, columns=['origin_port', 'X3pl', 'customs_procedures', 'logistic_hub', 'customer', 'material_handling'])


In [167]:
# Clean df dataset

df=df.drop(['product_id','order_id'],axis=1)
df = df.dropna()

df = pd.get_dummies(df, columns=['origin_port', 'X3pl', 'customs_procedures', 'logistic_hub', 'customer', 'material_handling'])

df['late_order']=df['late_order'].astype(int)

df['late_order_t'] = df['late_order']
df=df.drop(['late_order'],axis=1)

In [168]:
# Data Partioning

training_data, testing_data = train_test_split(df, test_size=0.05, random_state=25)

In [169]:
training_data=training_data.sample(frac=1)
train_nolate=training_data[training_data['late_order_t']==0]
train_late=training_data[training_data['late_order_t']==1]
train_nolate=train_nolate[:training_data['late_order_t'].value_counts()[1]]
print(train_nolate.shape)
print(train_late.shape)
training_data = pd.concat([train_nolate, train_late])
training_data = training_data.sample(frac=1)

(25787, 60)
(25787, 60)


In [170]:
def classificacio(clf, tuning=False):
  clf.fit(training_data.iloc[:,:-1], training_data.iloc[:,-1])
  preds = pd.Series(clf.predict(testing_data.iloc[:,:-1]), name='preds')
  reals = testing_data.iloc[:,-1]
  reals.name = 'reals'
  reals.index = range(len(preds))
  print('acc: ', accuracy_score(reals, preds))
  print('rec: ', recall_score(reals,preds, pos_label=1))
  print('prec: ',recall_score(reals,preds, pos_label=1))
  print('f1:', f1_score(reals, preds, pos_label=1))
  if tuning:
    print('best pars:', clf.best_params_)
    pd.crosstab(reals,preds)
  return [reals,preds,clf]


In [171]:
resultats1={}

In [172]:
resultats1['RandomForestClassifier']=classificacio(GradientBoostingClassifier())
pd.crosstab(resultats1['RandomForestClassifier'][0],resultats1['RandomForestClassifier'][1])

acc:  0.7556061667834618
rec:  0.7080838323353293
prec:  0.7080838323353293
f1: 0.5756008518405841


preds,0,1
reals,,
0,3367,1005
1,390,946


In [155]:
resultats1['RandomForestClassifier'][2].predict(test_df.iloc[:,:-1])

c:\Users\jordi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- logistic_hub_Direct shipment

  warnings.warn(message, FutureWarning)


ValueError: X has 58 features, but RandomForestClassifier is expecting 59 features as input.